<a target="_parent" href="https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/main/docs/notebooks/demo/navigator/text-to-code/text-to-python-evol.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 🎨 Navigator Data Designer SDK: Text-to-Python Evolution

This notebook demonstrates how to use the Gretel Navigator SDK to create a synthetic data generation pipeline for Python code examples, with a focus on evolutionary improvements. We'll build a system that generates Python code based on natural language instructions, validates it, analyzes issues, and then improves the code based on feedback.

In [1]:
%%capture
# Install the latest version of Gretel client and dependencies
%pip install -U gretel_client 

In [ ]:
from gretel_client.navigator_client import Gretel

# Initialize Gretel client and Data Designer
gretel = Gretel(api_key="prompt", endpoint="https://api.dev.gretel.ai")
aidd = gretel.data_designer.new(model_suite="apache-2.0")

## 🌱 Define Categorical Seed Columns

We'll set up our seed columns for industry sectors, code complexity, and instruction types. These will help generate diverse and relevant code examples.

In [ ]:
# Add industry sector categories
aidd.add_column(
    name="industry_sector",
    type="category",
    params={
        "values": ["Healthcare", "Finance", "Technology"],
        "description": "The industry sector for the code example"
    }
)

# Add topic as a subcategory of industry_sector
aidd.add_column(
    name="topic",
    type="subcategory",
    params={
        "category": "industry_sector",
        "values": {
            "Healthcare": [
                "Electronic Health Records (EHR) Systems",
                "Telemedicine Platforms", 
                "AI-Powered Diagnostic Tools"
            ],
            "Finance": [
                "Fraud Detection Software",
                "Automated Trading Systems",
                "Personal Finance Apps"
            ],
            "Technology": [
                "Cloud Computing Platforms",
                "Artificial Intelligence and Machine Learning Platforms",
                "DevOps and CI/CD Tools"
            ]
        }
    }
)

# Add code complexity with subcategory for code concepts
aidd.add_column(
    name="code_complexity",
    type="category",
    params={
        "values": ["Beginner", "Intermediate", "Advanced"],
        "description": "The complexity level of the code"
    }
)

# Add code_concept as a subcategory of code_complexity
aidd.add_column(
    name="code_concept",
    type="subcategory",
    params={
        "category": "code_complexity",
        "values": {
            "Beginner": [
                "Variables",
                "Data Types",
                "Functions",
                "Loops",
                "Classes"
            ],
            "Intermediate": [
                "List Comprehensions",
                "Object-oriented programming",
                "Lambda Functions",
                "Web frameworks",
                "Pandas"
            ],
            "Advanced": [
                "Multithreading",
                "Context Managers",
                "Generators"
            ]
        }
    }
)

# Add instruction phrases
aidd.add_column(
    name="instruction_phrase",
    type="category",
    params={
        "values": [
            "Write a function that",
            "Create a class that",
            "Implement a script",
            "Can you create a function",
            "Develop a module that"
        ],
        "description": "Starting phrase for the code instruction"
    }
)

## ✨ Define Initial Code Generation

First, we'll set up the columns for generating the instruction and initial code implementation using the same approach as in the text-to-python notebook.

In [ ]:
# Generate instruction for the code
aidd.add_column(
    name="instruction",
    type="llm-text",
    system_prompt="You are an expert at generating clear and specific programming tasks.",
    prompt="""\
Generate an instruction to create Python code that solves a specific problem. 
Each instruction should begin with one of the following phrases: {{instruction_phrase}}.

Important Guidelines:
* Industry Relevance: Ensure the instruction pertains to the {{industry_sector}} sector and {{topic}} topic.
* Code Complexity: Tailor the instruction to the {{code_complexity}} level. Utilize relevant {{code_concept}} where appropriate to match the complexity level.
* Clarity and Specificity: Make the problem statement clear and unambiguous. Provide sufficient context to understand the requirements without being overly verbose.
* Response Formatting: Do not include any markers such as ### Response ### in the instruction.
"""
)

# Generate the initial Python code
aidd.add_column(
    name="initial_code",
    type="llm-code",
    model_alias="code",
    output_format="python",
    system_prompt="You are an expert Python programmer who writes clean, efficient, and well-documented code.",
    prompt="""\
Write Python code for the following instruction:
Instruction: {{instruction}}

Important Guidelines:
* Code Quality: Your code should be clean, complete, self-contained and accurate.
* Code Validity: Please ensure that your python code is executable and does not contain any errors.
* Packages: Remember to import any necessary libraries, and to use all libraries you import.
* Complexity & Concepts: The code should be written at a {{code_complexity}} level, making use of concepts such as {{code_concept}}.
"""
)

## 🔍 Code Validation and Analysis

Now we'll add validation for the initial code and generate analysis of any issues found.

In [ ]:
# Validate the initial code
aidd.add_column(
    name="code_validation",
    type="code-validation",
    code_lang="python",
    target_column="initial_code"
)

In [ ]:
# Generate a detailed error analysis and improvement plan
aidd.add_column(
    name="code_analysis",
    type="llm-text",
    prompt="""\
Analyze the following Python code and its validation results:

INSTRUCTION:
{{instruction}}

INITIAL CODE:
{{initial_code}}

VALIDATION RESULTS:
{{code_validation}}

{% if code_validation != '[]' %}
Please provide:
1. A detailed analysis of each error or warning (categorize by type: convention, warning, error, refactor)
2. Specific recommendations that directly address each issue
3. A structured plan for implementing fixes while maintaining code functionality
4. Any PEP 8 style improvements that would improve code quality
{% else %}
The code passes all validation checks. Provide potential optimizations for:
1. Code readability
2. Performance improvements
3. Better adherence to Python best practices
4. Enhanced documentation
{% endif %}
"""
)

## 🔄 Code Evolution

Next, we'll create the improved version of the code based on the analysis and validation.

In [ ]:
# Generate improved code based on feedback
aidd.add_column(
    name="improved_code",
    type="llm-code",
    model_alias="code",
    output_format="python",
    system_prompt="You are an expert Python programmer focused on writing production-quality code that adheres to best practices.",
    prompt="""\
Rewrite and improve the following Python code based on the analysis provided.

ORIGINAL INSTRUCTION:
{{instruction}}

INITIAL CODE:
{{initial_code}}

CODE ANALYSIS:
{{code_analysis}}

Your task is to create a revised version that:
1. Addresses all issues identified in the analysis
2. Follows PEP 8 style guidelines systematically
3. Eliminates common anti-patterns
4. Includes comprehensive docstrings for functions, classes, and modules
5. Uses type hints for function parameters and return values where appropriate
6. Implements proper error handling with specific exception types
7. Ensures all imports are properly organized and used

The goal is production-quality code that would pass a professional code review at a {{code_complexity}} level.
"""
)

# Validate the improved code
aidd.add_column(
    name="improved_code_validation",
    type="code-validation",
    code_lang="python",
    target_column="improved_code"
)

## 📊 Evaluation

Finally, we'll add an evaluation that compares the initial and improved code.

In [ ]:
from gretel_client.data_designer.judge_rubrics import PYTHON_RUBRICS

# Add judge evaluation
aidd.add_column(
    name="code_judge_result", 
    type="llm-judge", 
    prompt=(
        "You are an expert in Python programming, with specialized knowledge in software engineering, "
        "data science, and algorithmic problem-solving. You think about potential flaws and errors "
        "in the code. You are a tough critic, but a fair one.\n\n"
        "Take a deep breath and use the Python Code Quality Rubric below to score the **Generated Python Code** "
        "based on the INSTRUCTIONS.\n\n"
        "#### INSTRUCTIONS\n"
        "The Generated Python Code should be a valid response to the Natural Language Prompt below\n\n"
        "Natural Language Prompt:\n"
        "{{ instruction }}\n\n"
        "Generated Python Code\n"
        "```python\n"
        "{{ improved_code }}\n"
        "```\n"
    ),
    rubrics=PYTHON_RUBRICS
)

## 👀 Generate Preview Dataset

Let's generate a preview to see how our evolved code examples look.

In [ ]:
# Generate a preview
preview = aidd.preview()

## 🔎 Easily inspect individual records

- Run the cell below to display individual records for inspection.
- Run the cell multiple times to cycle through the preview records.
- Alternatively, you can pass the `index` argument to `display_sample_record` to display a specific record.

In [ ]:
preview.display_sample_record()

## 🚀 Generate Full Dataset

If you're satisfied with the preview, you can generate a larger dataset using a batch workflow.

In [11]:
# # Submit batch job
# workflow_run = aidd.create(
#     num_records=50,
#     name="text_to_python_evolution",
#     wait_for_completion=True
# )
# print("\nGenerated dataset shape:", workflow_run.dataset.df.shape)

In [12]:
# # Download evaluation report
# path = workflow_run.download_report(wait_for_completion=True)